## GMM estimation of Lucas Tree Model

In [53]:
import numpy as np
from fredapi import Fred
import pandas as pd


fred = Fred(api_key='81d6202762edfdf9248639fbd6972c39')

C = (fred.get_series('PCECC96'))
R = fred.get_series('WILL5000INDFC')
Rf = fred.get_series('TB3MS')
Deflator = fred.get_series('GDPDEF')






#Computing quarterly observations for stock returns and Treasuries

In [54]:
import datetime
# Downsample to quarterly frequency
Rq = R.resample('Q', how='mean', fill_method='bfill') 
Rfq = Rf.resample('Q', how='mean', fill_method='bfill') 

# Shift dates
Rq = Rq.shift(1, freq='D')
Rfq = Rfq.shift(1, freq='D')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).mean().bfill()
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).mean().bfill()
  after removing the cwd from sys.path.


In [55]:
data = pd.DataFrame()
data.insert(0,"Consumption", C)
data.insert(1, "Returns", Rq)
data.insert(2, "Tbill", Rfq)
data.insert(3, "Deflator", Deflator)
print(data)

            Consumption     Returns     Tbill  Deflator
1947-01-01     1274.952         NaN  0.380000    11.960
1947-04-01     1296.163         NaN  0.380000    12.131
1947-07-01     1300.388         NaN  0.380000    12.335
1947-10-01     1300.801         NaN  0.736667    12.639
1948-01-01     1307.283         NaN  0.906667    12.739
1948-04-01     1322.494         NaN  0.990000    12.854
1948-07-01     1324.446         NaN  1.000000    13.092
1948-10-01     1335.016         NaN  1.050000    13.132
1949-01-01     1337.177         NaN  1.140000    13.062
1949-04-01     1357.672         NaN  1.170000    12.931
1949-07-01     1360.751         NaN  1.170000    12.872
1949-10-01     1380.702         NaN  1.043333    12.873
1950-01-01     1403.690         NaN  1.076667    12.853
1950-04-01     1426.807         NaN  1.103333    12.897
1950-07-01     1500.076         NaN  1.153333    13.177
1950-10-01     1454.855         NaN  1.220000    13.425
1951-01-01     1490.034         NaN  1.336667   

In [56]:
data.insert(4, "Real Returns", \
            (1 + data['Returns'].pct_change()*100)/ \
            (1 + data['Deflator'].pct_change()*100) - 1)

In [57]:
data.insert(4, "Real Tbill", \
            (1 + data['Tbill'].pct_change()*100)/ \
            (1 + data['Deflator'].pct_change()*100) - 1)

In [59]:
Dataset = data.dropna('index')

In [60]:
print(Dataset)

            Consumption     Returns     Tbill  Deflator  Real Tbill  \
1971-04-01     3185.993    1.080000  3.840000    22.685  -12.789307   
1971-07-01     3211.394    1.136667  4.250000    22.916    4.785621   
1971-10-01     3264.679    1.113333  5.010000    23.107    9.298648   
1972-01-01     3307.767    1.113333  4.230000    23.458   -6.783543   
1972-04-01     3370.736    1.240000  3.436667    23.604  -11.943692   
1972-07-01     3422.682    1.280000  3.770000    23.830    4.465907   
1972-10-01     3503.001    1.293333  4.220000    24.134    4.684547   
1973-01-01     3566.957    1.356667  4.863333    24.412    6.549074   
1973-04-01     3565.349    1.306667  5.700000    24.787    6.177694   
1973-07-01     3577.898    1.196667  6.603333    25.270    4.713878   
1973-10-01     3567.227    1.243333  8.323333    25.773    8.044450   
1974-01-01     3535.312    1.176667  7.500000    26.260   -4.077225   
1974-04-01     3548.041    1.130000  7.616667    26.880   -0.239645   
1974-0

In [83]:
# Compute the stochastic discount factor 
β = 0.99

def SDF(γ, Dataset):
    M = β*(1 + Dataset.Consumption.pct_change()*100)**(-γ)
    return M

def EulerError(γ, Dataset):
    EM  = np.mean(SDF(γ, Dataset))

    ER  = np.mean(Dataset['Real Returns'])

    R = np.array(Dataset['Real Returns'])
    M = np.array(SDF(γ,Dataset))
    covar = np.cov(M,R)[1,1]

    error = 1 - covar - EM*ER
    print(error)
    return error


EulerError(0.5,Dataset)


-18.517423212577743


-18.517423212577743

In [88]:
from scipy.optimize import fsolve

result = fsolve(EulerError,0.3, args=(Dataset))
print(result)

-18.58808677868204
-18.58808677868204
-18.58808677868204
-18.588086776833755
-8.20822769124614e+32
-18.58808677868204
[0.3]
